### testing the importance of distance on instruction priority

In [1]:
%pip install huggingface_hub
%pip install python-dotenv
%pip install huggingface
%pip install datasets
%pip install transformers
%pip install matplotlib
%pip install accelerate


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To updat

In [2]:
import os
from dotenv import load_dotenv
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["DATASETS_DISABLE_PROGRESS_BAR"] = "1"

load_dotenv()
huggingface_token = os.getenv("HUGGINGFACE_TOKEN")

In [3]:
!hf auth login --token {huggingface_token}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.


Token is valid (permission: fineGrained).
The token `a3` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `a3`


In [4]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import random
import torch
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import math

In [5]:
#load wiki
ds = load_dataset("wikitext", "wikitext-2-raw-v1")

In [6]:
print(type(ds))
print(ds)

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [7]:
print(ds["train"][7])

{'text': ' = = Gameplay = = \n'}


In [8]:
def keep_row(example):
    text = example['text']
    if not text:
        return False
    if text == "\n":
        return False
    # if text.startswith("="):
    #     return False
    return True

In [9]:
filtered_wiki = ds.filter(keep_row)

In [10]:
model_name = "google/gemma-2-9b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [11]:
### tokenize dataset
full_text = "\n".join([line for line in filtered_wiki["train"]["text"]])
all_tokens = tokenizer.encode(full_text, add_special_tokens = False)

In [12]:
def get_r_wiki_slice(n_tokens):
    start_idx = random.randint(0, len(all_tokens) - n_tokens - 1)
    return all_tokens[start_idx : start_idx + n_tokens]

In [13]:
system_prompt = "Your goal is to respond to the user's request while making sure his instructions are fully satistfied. An instruction will appear in the form 'Instruction: <INSTRUCTION>.'\n"

end_question = "\nNow, provide your response based on the previous instructions. \n"

instruction_set = {
    "language": ["Respond in French.", "Respond in German."],
    "first_word": ["Begin your response with the word 'CHERRY'.", "Begin your response with the word 'APPLE'."],
}


In [14]:
def find_token_span(target_ids, full_ids):
    n = len(target_ids)
    for i in range(len(full_ids) - n + 1):
        if full_ids[i : i + n] == target_ids:
            return i, i + n
    return None, None

In [15]:
def construct_single_instr_prompt(instr_text, pos_pct, total_length=2048):
    formatted_instr = f"\n\nInstruction: {instr_text}\n\n"
    
    dummy_messages = [{"role": "user", "content": "DUMMY"}]
    dummy_ids = tokenizer.apply_chat_template(dummy_messages, tokenize=True, add_generation_prompt=True)
    template_overhead = len(dummy_ids) - len(tokenizer.encode("DUMMY", add_special_tokens=False))
    
    instr_tokens = tokenizer.encode(formatted_instr, add_special_tokens=False)
    sys_tokens = tokenizer.encode(system_prompt, add_special_tokens=False)
    end_tokens = tokenizer.encode(end_question, add_special_tokens=False)
    
    fixed_parts_len = len(sys_tokens) + len(instr_tokens) + len(end_tokens) + template_overhead
    filler_budget = total_length - fixed_parts_len
    
    if filler_budget < 0:
        raise ValueError(f"Instructions too long! Overhead is {fixed_parts_len}")

    filler_tokens = get_r_wiki_slice(filler_budget)
    split_idx = int(pos_pct * filler_budget)
    
    user_content = (
        system_prompt + 
        tokenizer.decode(filler_tokens[:split_idx]) + 
        formatted_instr + 
        tokenizer.decode(filler_tokens[split_idx:]) + 
        end_question
    )
    
    messages = [{"role": "user", "content": user_content}]
    input_ids = tokenizer.apply_chat_template(
        messages, 
        tokenize=True, 
        add_generation_prompt=True, 
        return_tensors="pt"
    )

    if input_ids.shape[1] != total_length:
        diff = input_ids.shape[1] - total_length
        if diff > 0:

            input_ids = torch.cat([input_ids[:, :10], input_ids[:, 10+diff:]], dim=1)
        else:
            padding = torch.full((1, abs(diff)), tokenizer.pad_token_id).to(input_ids.device)
            input_ids = torch.cat([input_ids[:, :10], padding, input_ids[:, 10:]], dim=1)

    return input_ids, None

In [16]:
test_ids, _ = construct_single_instr_prompt("Begin with CHERRY", 0.5, 1024)
print(f"Total length: {test_ids.shape[1]}")
print(f"End of prompt tokens: {test_ids[0, -10:].tolist()}")
print(f"End of prompt decoded: '{tokenizer.decode(test_ids[0, -15:])}'")

Total length: 1024
End of prompt tokens: [611, 573, 4509, 12027, 235265, 107, 108, 106, 2516, 108]
End of prompt decoded: ', provide your response based on the previous instructions.<end_of_turn>
<start_of_turn>model
'


In [17]:
cherry_ids = [
    tokenizer.encode(" CHERRY", add_special_tokens=False)[0],
    tokenizer.encode("CHERRY\n", add_special_tokens=False)[0],
    tokenizer.encode("CHERRY ", add_special_tokens=False)[0],
    tokenizer.encode("CHERRY", add_special_tokens=False)[0],
    tokenizer.encode("\nCHERRY", add_special_tokens=False)[0],
]

In [ ]:
def batched_position_sweep(model, instr_text, total_length, samples_per_pos=100, batch_size=2):
    device = next(model.parameters()).device
    pcts = np.linspace(0.0, 1.0, 21)
    target_token_id = tokenizer.encode(" CHERRY", add_special_tokens=False)[0]
    
    results = {round(pct, 2): {"logits": [], "is_correct": []} for pct in pcts}
    # This will now store a LIST of completions for each position
    decoded_completions = {round(pct, 2): [] for pct in pcts}
    
    model.eval()
    for pct in pcts:
        pct_key = round(pct, 2)
        print(f"Processing Position: {pct_key:.2f}")
        
        num_batches = (samples_per_pos + batch_size - 1) // batch_size
        
        # Determine which specific global indices we want to decode (5%)
        num_to_decode = math.ceil(0.05 * samples_per_pos)
        decode_indices = set(random.sample(range(samples_per_pos), num_to_decode))
        
        global_idx = 0 

        for b in tqdm(range(num_batches)):
            batch_list = []
            for _ in range(batch_size):
                if len(batch_list) + b*batch_size < samples_per_pos: # Handle final uneven batch
                    ids, _ = construct_single_instr_prompt(instr_text, pct, total_length)
                    batch_list.append(ids)
            
            batch_tensor = torch.cat(batch_list, dim=0).to(device)
            
            with torch.no_grad():
                outputs = model(batch_tensor)
                last_token_logits = outputs.logits[:, -1, :] 
            
            # Logit/Accuracy Data
            cherry_logits = last_token_logits[:, target_token_id].cpu().tolist()
            results[pct_key]["logits"].extend(cherry_logits)
            
            predictions = torch.argmax(last_token_logits, dim=-1)
            correct_mask = [p.item() in cherry_ids for p in predictions]
            results[pct_key]["is_correct"].extend(correct_mask)
            

            # decoded_batch = tokenizer.batch_decode(gen_ids[:, total_length:], skip_special_tokens=True)
            # is_correct = ["CHERRY" in resp.upper() for resp in decoded_batch]
            # results[pct_key]["is_correct"].extend(is_correct)

            # --- THE SANITY CHECK DECODE ---
            # We check each item in the current batch to see if its global index is in our 5% set
            for i in range(len(batch_list)):
                if global_idx in decode_indices:
                    with torch.no_grad():
                        # Generate for just this one sequence
                        gen_ids = model.generate(
                            batch_tensor[i:i+1], 
                            max_new_tokens=15, # Increased to see the full context of the failure
                            do_sample=False,
                            pad_token_id=tokenizer.eos_token_id
                        )
                        # Extract and store completion only
                        completion = tokenizer.decode(gen_ids[0, total_length:], skip_special_tokens=True)
                        decoded_completions[pct_key].append(completion)
                
                global_idx += 1
            
            del outputs
            torch.cuda.empty_cache()
            
    return results, decoded_completions

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="eager" 
)

`torch_dtype` is deprecated! Use `dtype` instead!


In [22]:
def plot_dual_results(results):
    x = sorted(results.keys())
    
    # Prepare Data
    logit_means = [np.mean(results[pct]["logits"]) for pct in x]
    logit_stds = [np.std(results[pct]["logits"]) for pct in x]
    accuracy = [np.mean(results[pct]["is_correct"]) for pct in x]
    
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Plot Logits
    color = 'tab:blue'
    ax1.set_xlabel('Position (0.0=Start, 1.0=End)')
    ax1.set_ylabel('Logit Magnitude (CHERRY)', color=color)
    ax1.errorbar(x, logit_means, yerr=logit_stds, fmt='-o', color=color, label='Logit')
    ax1.tick_params(axis='y', labelcolor=color)

    # Plot Accuracy on second axis
    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.set_ylabel('Accuracy (Was CHERRY Top-1)', color=color)
    ax2.plot(x, accuracy, '-s', color=color, label='Accuracy')
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.set_ylim(-0.1, 1.1) # Accuracy is 0 to 1

    plt.title(f"Instruction Salience:")
    fig.tight_layout()
    plt.grid(True, alpha=0.3)
    plt.show()

# --- Execution ---
# Start with a shorter haystack to confirm the trend fast
sweep_data, examples = batched_position_sweep(model, instruction_set["first_word"][0], total_length=6144, samples_per_pos=20)
plot_dual_results(sweep_data)

Processing Position: 0.00


  0%|          | 0/10 [00:03<?, ?it/s]


NameError: name 'decoded_batch' is not defined

In [ ]:
print(examples)

{0.0: ['CHERRY  This is a biography of Wilhelm Busch, a German author and', 'CHERRY  This is a text about actor and comedian Adrian Dunbar and his', 'CHERRY  The provided text discusses various aspects of stars, including their luminosity'], 0.05: ['CHERRY  The Qedarites were a nomadic Arab tribal confederation that', 'CHERRY  \n\nGregory Helms, better known by his wrestling personas "The', 'CHERRY The ships had a tertiary armament for protection against torpedo boats in the'], 0.1: ['CHERRY  The 3rd Light Horse Brigade halted at Hill 7', 'CHERRY  The text you provided describes the history of the Réunion solitaire,', 'CHERRY  The text you provided describes the rules of the key in basketball'], 0.15: ['CHERRY  The NTSB determined that the probable cause of the accident was', 'CHERRY  Anderson has been widely praised for his filmmaking talent, with many', 'CHERRY  This is a fascinating biography of R.C.  Wheeler'], 0.2: ['CHERRY  The Battle of Hubbardton was a significant engagement in

In [21]:
import gc, torch
for name in ("trainer","model","optim","scheduler"):
    if name in globals(): del globals()[name]
gc.collect()
torch.cuda.empty_cache()